In [1]:
import re
import datetime
import pandas as pd
import requests 
import os

In [17]:
#pip install mysql-connector-python
# pip install python-dotenv
import mysql.connector
conexao = mysql.connector.connect(host='localhost',user='root',password='',database='producao')
cursor = conexao.cursor()

In [4]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI

# pip install openai
# pip install langchain-openai
# saldo disponível
# https://platform.openai.com/settings/organization/billing/overview
# chaves API-KEY
# https://platform.openai.com/api-keys

In [5]:
load_dotenv(dotenv_path='.env', override=True)

True

In [6]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
question = "Qual a capital do Brasil?"
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key = openai_api_key, temperature=0.7,max_tokens=256)
messages=[{"role":"user", "content": question}]
response = llm.invoke(messages)
print(response.content)

A capital do Brasil é Brasília.


In [18]:
# pip install tabulate
numero = "PI0808715"
# atualiza no localhost as tres tabelas: carga, anterioridades e anterioridades_desc
comando = f"SELECT * FROM arquivados WHERE numero='{numero}'"
cursor.execute(comando)
resultado = cursor.fetchall()
df = pd.DataFrame(resultado)
print(df.to_markdown(index=False)) 

|       0 | 1             | 2         | 3          | 4     |   5 |   6 |
|--------:|:--------------|:----------|:-----------|:------|----:|----:|
| 1011532 | 1.1           | PI0808715 | 2011-08-09 | dialp |   0 |   0 |
| 1475660 | 1.3           | PI0808715 | 2014-08-12 | dialp |   0 |   0 |
| 1501853 | 6.6           | PI0808715 | 2014-09-16 | dialp |   0 |   0 |
| 2788968 | 7.1           | PI0808715 | 2017-04-18 | dialp |   0 |   1 |
| 2790019 | 15.11         | PI0808715 | 2017-04-18 | dialp |   0 |   0 |
| 2956330 | 9.2           | PI0808715 | 2017-09-19 | dialp |   0 |   2 |
| 3015281 | 12.2          | PI0808715 | 2017-12-19 | dialp |   0 |   0 |
| 4414189 | PR - Recursos | PI0808715 | 2022-01-04 | dialp |   0 |   0 |
| 4551344 | PR - Recursos | PI0808715 | 2022-08-16 |       |   0 |   0 |


In [9]:
df

,0,1,2,3,4,5,6
0,1011532,1.1,PI0808715,2011-08-09,dialp,0,0
1,1475660,1.3,PI0808715,2014-08-12,dialp,0,0
2,1501853,6.6,PI0808715,2014-09-16,dialp,0,0
3,2788968,7.1,PI0808715,2017-04-18,dialp,0,1
4,2790019,15.11,PI0808715,2017-04-18,dialp,0,0
5,2956330,9.2,PI0808715,2017-09-19,dialp,0,2
6,3015281,12.2,PI0808715,2017-12-19,dialp,0,0
7,4414189,PR - Recursos,PI0808715,2022-01-04,dialp,0,0
8,4551344,PR - Recursos,PI0808715,2022-08-16,,0,0


In [10]:
for item in resultado:
    numero = item[0]
    despacho = item[1]
    print(f"Número: {numero} | Despacho: {despacho}")

Número: 1011532 | Despacho: 1.1
Número: 1475660 | Despacho: 1.3
Número: 1501853 | Despacho: 6.6
Número: 2788968 | Despacho: 7.1
Número: 2790019 | Despacho: 15.11
Número: 2956330 | Despacho: 9.2
Número: 3015281 | Despacho: 12.2
Número: 4414189 | Despacho: PR - Recursos
Número: 4551344 | Despacho: PR - Recursos


In [21]:
comando = f"SELECT * FROM titulo2 WHERE titulo_cepit<>'' and resumo_cepit like '%Abstract%'"
cursor.execute(comando)
resultado = cursor.fetchall()
df = pd.DataFrame(resultado)
total = 0

for item in resultado:
    total = total + 1
    if (total<1000):
        continue
    if (total>2000):
        break
    numero = item[1]
    titulo_cepit = item[5]
    resumo_cepit = item[6]
    #print(f"Título: {titulo_cepit}")
    #print(f"Resumo: {resumo_cepit}\n")

    question = f"""Elimine os trechos em inglês. Converta para minúsculas mantendo a primeira letra de cada frase em maiúsculo.
        Elimine os termos 'Resumo' e 'Patente de invenção' no início. Siglas devem ser mantidas na forma original, por exemplo 
        PBCH. Mostre como resposta apenas o texto em português. Se todo o texto estiver em inglês não mostre nada: O texto a analisar é o seguinte: {resumo_cepit}"""
    messages=[{"role":"user", "content": question}]
    response = llm.invoke(messages)
    #print("\033[34m" + response.content + "\033[0m")
    resumo = response.content
    resumo = resumo.replace('"', '')
    resumo = resumo.replace("'", '')
    cmd = f"update titulo set resumo_cepit='{resumo}' where numero='{numero}'"
    print(cmd + ';')
    #print('#################')

update titulo set resumo_cepit='Tampa de abandono submarina de longo prazo, moldada, testável, para cabos elétricos e método de produção. A presente invenção refere-se a um conjunto de cabo umbilical, que é criado por um processo que, em uma modalidade, ajusta uma tampa de abandono sobre uma extremidade de um cabo eletricamente condutor, bem como um disco de polietileno, no qual um conjunto de pinos de conexão foram pré-moldados. A tampa de abandono é moldada por injeção sobre a extremidade do cabo eletricamente condutor, onde um volume cilíndrico de polietileno simultaneamente moldado por injeção para o disco de polietileno. Em determinadas modalidades, alguns dos pinos de conexão estão conectados no núcleo de energia condutor, umbilical.' where numero='112015004614';
update titulo set resumo_cepit='Implantes ósseos de compósito. A presente invenção refere-se a implantes ósseos multicompósitos naturais como implantes osso-tecido conectivo-osso e osteocondrais para a substituição e/ou 

update titulo set resumo_cepit='Trata-se de um aparelho para determinar a capacidade de coagulação do sangue que compreende um atuador para mover cilindricamente um membro dentro de uma amostra de sangue recebida em uma cavidade em uma bandeja, e um dentre um sensor de deflexão e um sensor de posição para determinar a posição do membro umedecido mediante a atuação pelo atuador. A posição teórica do membro umedecido, conforme determinado com o uso de dados físicos de membro umedecido e uma força de atuador conhecida é comparada com a posição ou deflexão captada do membro umedecido, e a resistência ao movimento do membro umedecido causada pelo sangue é determinada e correlacionada a uma capacidade de coagulação.' where numero='112015005288';
update titulo set resumo_cepit='MÉTODO E APARELHO PARA DECORAR UM PAINEL. Trata-se de um método de decoração de um painel (7) que compreende as etapas de suprir um painel (7) para um transportador (3), executar pelo menos uma primeira etapa de impres

update titulo set resumo_cepit='A presente invenção fornece várias formulações para distribuição oral de peptídeos angiotensina.' where numero='112015005738';
update titulo set resumo_cepit='Método para a intensificação de imunoterapias específicas no tratamento de câncer. Os métodos e composições da invenção referem-se à intensificação das imunoterapias específicas no tratamento do câncer. Em particular, os aspectos da invenção referem-se às novas abordagens para elevar a função imunológica utilizando um composto farmacêutico de carboidrato complexo isoladamente ou em combinação com outra imunoterapia direcionada para aumentar a eficácia da imunoterapia de câncer.' where numero='112015005772';
update titulo set resumo_cepit='A presente invenção refere-se um método para processar um desconto em tempo real em um ponto de vendas que inclui: armazenar, em um banco de dados, uma pluralidade de negócios, cada negócio incluindo uma identificação de consumidor e um modificador de transação, r

update titulo set resumo_cepit='A presente invenção se refere a um aparelho de sensor vestível sem fio. O aparelho de sensor vestível sem fio inclui uma plataforma de sensor que tem um dispositivo de processamento de sinal com um mecanismo computacional para implantar tarefas de processamento de sinal. A plataforma de sensor é configurada para receber sinais de pelo menos um sensor acoplado a isso. Um circuito de comunicação sem fio é acoplado à plataforma de sensor. O circuito de comunicação sem fio compreende um controlador-mestre de enlace configurado para se comunicar com um dispositivo sem fio e transferir dados. Em um aspecto, o controlador-mestre de enlace é configurado para controlar a transmissão de dados por um enlace de comunicação estabelecido com o dispositivo sem fio, que compreende o controle de temporização e o controle de frequência. O sensor vestível sem fio pode incluir um processador, uma memória e um acelerômetro acoplado à plataforma de sensor.' where numero='1120

update titulo set resumo_cepit='Trata-se de um dispositivo de conversão de potência CA direta que adota uma câmara de elevador, em que a corrente-pico que flui através de um indutor que a câmara de elevador tem, é reduzida. Quando um serviço de retificação (dr), um serviço de descarga (dc), a tensão (Vc) ao longo de um capacitor (34) e a tensão de retificação (Vrec) de uma tensão de CA (Vin) são introduzidos, a tensão de enlace de CC virtual (Vdc) em um inversor (6) é representada por dcVc+drVrec. O serviço de descarga (dc) é a proporção de tempo na qual um comutador (41) está conduzindo. O serviço de retificação (dr) tem o valor da subtração do serviço de descarga (dc) e um serviço de tensão zero (dz) de 1. O serviço de tensão zero (dz) é a proporção de tempo na qual o inversor (6) está adotando um vetor de tensão zero independentemente da magnitude da tensão que o inversor (6) está emitindo. Em uma parte do período durante o qual a tensão de enlace de CC virtual (V' where numero='112

...

update titulo set resumo_cepit='Controlador de nivelador de volume e método de controle. São divulgados um controlador de nivelador de volume e um método de controle. Em uma modalidade, um controlador de nivelador de volume inclui um classificador de conteúdo de áudio para identificar o tipo de conteúdo de um sinal de áudio em tempo real, e uma unidade de ajuste para ajustar um nivelador de volume de maneira contínua com base no tipo de conteúdo tal como identificado. A unidade de ajuste pode ser configurada para correlacionar positivamente o ganho dinâmico do nivelador de volume com tipos informativos de conteúdo do sinal de áudio, e correlacionar negativamente o ganho dinâmico do nivelador de volume com tipos interferentes de conteúdo de sinal de áudio.' where numero='112015024037';
update titulo set resumo_cepit='A presente invenção divulga uma composição estereocomplexa de ácido poliláctico contendo cristais estereocomplexos puros e um processo para a sua fabricação. A composição d

update titulo set resumo_cepit='A presente invenção refere-se a um dispositivo para controlar e isolar uma ferramenta (3) para isolar áreas em uma cavidade (1) que compreende: - um conduíte de entrada principal (C) que se comunica com o interior do dito compartimento (2), e que é obturado por um primeiro elemento (5) capaz de formar uma barreira a um fluido que circula no dito compartimento (2), enquanto rende além de uma primeira pressão (P1), - esse conduíte (C) se comunica com dois conduítes auxiliares (6, 8), as extremidades dos quais se unem a fim de formar um conduíte de saída (9), - um (8) desses conduítes auxiliares que forma uma primeira câmara, enquanto, no outro (6), um segundo elemento (7) é fornecido que forma uma barreira móvel entre uma primeira posição e uma segunda posição, caracterizado pelo fato de que na dita primeira posição, o dito segundo elemento (7) que forma uma barreira e a parede (630) do dito segundo conduíte auxiliar (6) fazem entre eles uma câmara anular 

update titulo set resumo_cepit='Sistema dispensador de mercadorias a granel. A presente invenção se refere a um dispensador de mercadorias a granel para a dispensação de mercadorias que estão contidas em sacos grandes. O dispensador de mercadorias a granel inclui um suporte, um contentor de mercadorias a granel e um conjunto dispensador.' where numero='112015025743';
update titulo set resumo_cepit='Um análogo de nucleosídeo deuteratado da fórmula e os sais farmaceuticamente aceitáveis deste são proporcionados por esta revelação. A revelação também inclui composições farmacêuticas compreendendo um composto ou sal da fórmula e um transportador. Os compostos e sais da fórmula são úteis para tratamento de infecções virais, incluindo infecções de HCV. Um método para tratamento de um hospedeiro afligido com hepatite C ou outros distúrbios é também apresentado que inclui administrar uma quantidade efetiva de tratamento de um nucleosídeo ou nucleotídeo que tem deutério com pelo menos 50% de en

update titulo set resumo_cepit='A presente invenção refere-se a métodos para desenvolver células T criadas para imunoterapia e mais especificamente a métodos para modificar células T inativando-se em genes do checkpoint imunes, preferivelmente pelo menos dois selecionados a partir de diferentes séries de reação, para aumentar a atividade imune da célula T. Este método envolve o uso de endonucleases de corte raro específicas, em particular TALE-nucleases e polinucleotídeos codificando tais polipeptídeos, para precisamente alvejar uma seleção de genes fundamentais em células T, que estão disponíveis a partir de doadores ou a partir de cultura de células primárias. A invenção abre o caminho para estratégias de imunoterapia adotivas altamente eficientes para tratar o câncer e infecções virais.' where numero='112015028483';
update titulo set resumo_cepit='Métodos para modificar geneticamente células T alogênicas e altamente ativas para imunoterapia. A presente invenção refere-se a métodos p

update titulo set resumo_cepit='Um sistema separador de óxido de propileno que compreende uma coluna de destilação, um decantador e um sistema de lavagem com água. A coluna de destilação configurada para receber um fluxo de óxido de propileno bruto, descarregar um fluxo de impurezas compreendendo metanol e água e descarregar um fluxo inferior compreendendo a maior parte do óxido de propileno introduzido no fluxo de óxido de propileno bruto. O decantador é configurado para receber pelo menos parte do fluxo de impureza e um solvente hidrocarboneto para proporcionar a formação, no decantador, de uma fase orgânica e uma fase aquosa. A fase orgânica compreende óxido de propileno e um solvente hidrocarboneto e é enviada à coluna de destilação. A fase aquosa compreende predominantemente metanol por peso e água que entra no fluxo de impurezas. O sistema de lavagem com água é configurado para receber e purgar a fase aquosa do sistema de separação de óxido de propileno.' where numero='1120160014